In [1]:
import yt_dlp
import concurrent.futures

In [2]:
def download_youtube_video(url, save_path="./", cookies_path="cookies.txt"):
    try:
        ydl_opts = {
            'outtmpl': f"{save_path}/%(title)s.%(ext)s",
            'format': 'bestvideo+bestaudio/best',
            # 'cookies': cookies_path  # Provide the cookies file
            'cookiesfrombrowser': ('chrome',)
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Download completed!")
    except Exception as e:
        print(f"An error occurred: {e}")

In [3]:
def get_latest_videos_from_channel(channel_url, playlist_items = '1-2'):
    ydl_opts = {
        'quiet': False,  # 显示调试信息，帮助我们理解问题
        'extract_flat': True,  # 只提取视频信息，不下载视频
        'playlist_items': playlist_items, 
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            # 提取频道信息
            info_dict = ydl.extract_info(channel_url, download=False)

            # 获取视频条目列表
            videos = info_dict.get('entries', [])

            if not videos:
                print("No videos found.")
                return []

            # 提取所有视频链接
            video_links = [
                video['url'] for video in videos
                if video.get('duration', 0) <= 600  # 600秒等于10分钟
            ]
            
            return video_links

        except Exception as e:
            print(f"Error occurred: {e}")
            return []

In [4]:
def download_latest_videos_concurrently(channel_url, playlist_items='1-2'):
    """获取并并行下载频道的最新视频"""
    video_urls = get_latest_videos_from_channel(channel_url, playlist_items)
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(download_youtube_video, video_url) for video_url in video_urls]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"An error occurred during download: {e}")

In [6]:

channel_url = 'https://www.youtube.com/@NBA/videos'
download_latest_videos_concurrently(channel_url, playlist_items='1-2')

[youtube:tab] Extracting URL: https://www.youtube.com/@NBA/videos
[youtube:tab] @NBA/videos: Downloading webpage
[download] Downloading playlist: NBA - Videos
[youtube:tab] Playlist NBA - Videos: Downloading 2 items
[download] Downloading item 1 of 2
[download] Downloading item 2 of 2
[download] Finished downloading playlist: NBA - Videos
Extracting cookies from chrome
Extracted 2893 cookies from chrome
[youtube] Extracting URL: https://www.youtube.com/watch?v=7LYnNhyUfgg
[youtube] 7LYnNhyUfgg: Downloading webpage
[youtube] 7LYnNhyUfgg: Downloading ios player API JSON
[youtube] 7LYnNhyUfgg: Downloading mweb player API JSON
[youtube] 7LYnNhyUfgg: Downloading player 89dfc5b3
[youtube] 7LYnNhyUfgg: Downloading m3u8 information
[info] 7LYnNhyUfgg: Downloading 1 format(s): 248+251
[download] Destination: .//NBA’s Top 10 Plays of the Night ｜ November 20, 2024.f248.webm
[download] 100% of   34.08MiB in 00:00:06 at 5.61MiB/s   
[download] Destination: .//NBA’s Top 10 Plays of the Night ｜ Novem